In [2]:
import pandas as pd 
import json
import ast
import pandas as pd 
import sqlite3
import sys
sys.path.insert(0, '../util')
import util_refactorings as ur
from util_label import get_split_and_matched_dfs, label_sentences_add, get_sentence_and_doc_labels

In [7]:
doc_data = pd.read_csv('../../controlled-sequence-gen/data/doc-edits-large__add-labels.tsv', sep='\t', header=None)
idx = doc_data[2]

# Can't Use Orig Crowdsourcing Data -- Too much is in the training dataset

In [6]:
with open('../evaluation/data/data-downsampled.json') as f:
    balanced_output = json.load(f)

In [23]:
idx = idx.str.replace('.0', '', regex=False)

In [22]:
balanced_idx = (pd.DataFrame(balanced_output)
 .T.reset_index()['index']
 .apply(ast.literal_eval)
 .apply(lambda x: '%s-%s-%s' % (x[0], x[1], x[2]))
)

In [25]:
balanced_idx.isin(idx).value_counts()

True     267
False    133
Name: index, dtype: int64

# Create New Balanced Data

In [2]:
all_output_df = pd.read_pickle('cache/all-short-documents-eval-processed.pkl')
count_df = pd.read_csv('../modeling/data/training_data_short_15.csv.gz')

/Users/alex/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
count_df = count_df.set_index(['source', 'entry_id', 'version'])
count_df_sums = (
    count_df
    .assign(num_sents=1)
    .assign(refactored_up=lambda df: (df['refactored_label'] < 0).astype(int))
    .assign(refactored_down=lambda df: (df['refactored_label'] > 0).astype(int))
    .drop(['sent_idx', 'sentence'], axis=1)
    .groupby(level=[0, 1, 2])
    .sum()
)

all_output_df_w_counts = pd.merge(
    left=count_df_sums.reset_index(),
    right=all_output_df.reset_index(),
    left_on=['source', 'entry_id', 'version'],
    right_on=['source', 'entry_id', 'version_x']
).set_index(['source', 'entry_id', 'version_x', 'version_y']).drop(['version'], axis=1)

all_output_df_w_counts = (all_output_df_w_counts
 .loc[lambda df: (df['num_sents'] > 5) & (df['num_sents'] < 10)]
)

add_above_weights = (
    all_output_df_w_counts['add_above_label']
    .value_counts()
    .sort_index()
    .pipe(lambda s: 1/s)
    .to_frame('add_weight')
    .reset_index()
)

delete_weights = (
    all_output_df_w_counts['deleted_label']
    .value_counts()
    .sort_index()
    .pipe(lambda s: 1/s)
    .to_frame('del_weight')
    .reset_index()

)

refactored_weights = (
    all_output_df_w_counts['refactored_up']
    .value_counts()
    .sort_index()
    .pipe(lambda s: 1/s)
    .to_frame('ref_weight')
    .reset_index()
)

source_weights = (
    all_output_df_w_counts
    .reset_index()
    ['source']
    .value_counts()
    .sort_index()
    .pipe(lambda s: 1/s)
    .to_frame('source_weight')
    .reset_index()
)

In [4]:
all_output_df_w_counts = (all_output_df_w_counts
 .assign(key=lambda df: 
         df.reset_index()
         .apply(lambda x: '%s-%s-%s' % (x['source'], x['entry_id'], x['version_x']), axis=1)
         .values
        )
)

In [8]:
all_output_df_w_counts.loc[lambda df: ~df['key'].isin(idx)]

deleted_label  add_above_label  \
source entry_id version_x version_y                                   
ap     5        6         7                    4.0              5.0   
                15        16                   9.0              0.0   
       14       15        16                   8.0              0.0   
       17       13        14                   4.0              4.0   
                14        15                   8.0              0.0   
...                                            ...              ...   
wp     1918327  0         1                    0.0              3.0   
       1925996  0         1                    0.0              0.0   
       1926724  0         1                    5.0              7.0   
       1927548  0         1                    5.0              2.0   
       1946717  0         1                    0.0              0.0   

                                     add_below_label  edited_label  \
source entry_id version_x version_y                                  
ap     5        6         7                      5.0           4.0   
                15        16                     0.0           0.0   
       14       15        16                     7.0           0.0   
       17       13        14                     4.0           3.0   
                14        15                     8.0           0.0   
...                                              ...           ...   
wp     1918327  0         1                      3.0           2.0   
       1925996  0         1                      0.0           2.0   
       1926724  0         1                      0.0           0.0   
       1927548  0         1                      2.0           1.0   
       1946717  0         1                      0.0           2.0   

                                     unchanged_label  refactored_label  \
source entry_id version_x version_y                                      
ap     5        6         7                      1.0               0.0   
                15        16                     0.0               0.0   
       14       15        16                     1.0               0.0   
       17       13        14                     2.0               0.0   
                14        15                     1.0               0.0   
...                                              ...               ...   
wp     1918327  0         1                      6.0               0.0   
       1925996  0         1                      7.0               0.0   
       1926724  0         1                      1.0               0.0   
       1927548  0         1                      3.0               0.0   
       1946717  0         1                      7.0               0.0   

                                     num_sents  refactored_up  \
source entry_id version_x version_y                             
ap     5        6         7                  9              0   
                15        16                 9              0   
       14       15        16                 9              0   
       17       13        14                 9              0   
                14        15                 9              0   
...                                        ...            ...   
wp     1918327  0         1                  8              0   
       1925996  0         1                  9              0   
       1926724  0         1                  6              0   
       1927548  0         1                  9              0   
       1946717  0         1                  9              0   

                                     refactored_down  \
source entry_id version_x version_y                    
ap     5        6         7                        0   
                15        16                       0   
       14       15        16                       0   
       17       13        14                       0   
                14        15                       0   
.

In [9]:
ref_bal = (
    all_output_df_w_counts
             .reset_index()
             .loc[lambda df: ~df['key'].isin(idx)]
             # downsample for refactoring
             .merge(refactored_weights, left_on='refactored_up', right_on='index')
             .pipe(lambda df: df.sample(4_000, weights=df['ref_weight']))
             #  source weights
             .merge(source_weights, left_on='source', right_on='index')
             .assign(source_weight=lambda df: df.apply(lambda x: x['source_weight'] if x['refactored_up'] == 0 else x['source_weight'], axis=1))
             .pipe(lambda df: df.sample(2_000, weights=df['source_weight']))     
             .set_index(['source', 'entry_id', 'version_x', 'version_y'])
)

In [14]:
add_bal = (
    all_output_df_w_counts
     .reset_index()
     .loc[lambda df: ~df['key'].isin(idx)]
     # add_above_weights    
     .merge(add_above_weights, left_on='add_above_label', right_on='index')
     .pipe(lambda df: df.sample(50_000, weights=df['add_weight']))
     # source weights
     .merge(source_weights, left_on='source', right_on='index')
     .pipe(lambda df: df.sample(40_000, weights=df['source_weight']))     
     .set_index(['source', 'entry_id', 'version_x', 'version_y'])
)

In [15]:
del_bal = (
    all_output_df_w_counts
     .reset_index()
     .loc[lambda df: ~df['key'].isin(idx)]
     # delete weights
     .merge(delete_weights, left_on='deleted_label', right_on='index')
     .pipe(lambda df: df.sample(4_000, weights=df['del_weight']))
     #  source weights
     .merge(source_weights, left_on='source', right_on='index')
     .pipe(lambda df: df.sample(2_000, weights=df['source_weight']))     
     .set_index(['source', 'entry_id', 'version_x', 'version_y'])
)

In [16]:
from tqdm.auto import tqdm

In [17]:
def get_labels(df_to_transform):
    all_orig_sent_labels = []
    all_orig_doc_labels = []

    for idx in tqdm(range(len(df_to_transform))):
        df = df_to_transform.iloc[idx]
        ## orig data
        doc_sentences = df['nodes']
        x_ver = min(list(map(lambda x: x['version'], doc_sentences)))
        doc_sentences = list(filter(lambda x: x['version'] == x_ver, doc_sentences))
        doc_sents = pd.DataFrame(doc_sentences)
        e_id = df.name[1]
        doc = pd.DataFrame(df['arcs'])
        doc['entry_id'] = e_id
        doc_sents['entry_id'] = e_id
        orig_sents, orig_doc = get_sentence_and_doc_labels(doc, doc_sents)
        all_orig_sent_labels.append(orig_sents)
        all_orig_doc_labels.append(orig_doc)
        
    return pd.concat(all_orig_sent_labels), pd.concat(all_orig_doc_labels)

In [50]:
ref_sent_labels_df, ref_doc_labels_df  = get_labels(ref_bal)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [54]:
ref_doc_labels_df.to_csv('../modeling/data/doc-eval__ref-balanced.csv')
ref_sent_labels_df.to_csv('../modeling/data/sent-eval__ref-balanced.csv')

In [18]:
add_sent_labels_df, add_doc_labels_df  = get_labels(add_bal)

  0%|          | 0/40000 [00:00<?, ?it/s]

In [19]:
add_sent_labels_df.to_csv('../modeling/data/sent-eval__add-balanced-large.csv')
add_doc_labels_df.to_csv('../modeling/data/doc-eval__add-balanced-large.csv')

In [56]:
del_sent_labels_df, del_doc_labels_df = get_labels(del_bal)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [102]:
del_sent_labels_df.to_csv('../modeling/data/sent-eval__del-balanced.csv')
del_doc_labels_df.to_csv('../modeling/data/doc-eval__del-balanced.csv')
# del_sent_labels_df[['deleted_label', 'edited_label', 'unchanged_label']]

In [106]:
add_doc_labels_df

,,num_deleted,num_added,num_edited,num_refactored,sentences
entry_id,version,,,,,
996666,0,0,3,2,0,"Two Japanese mountain climbers caught in a snowstorm and buried under nearly a half-century of Alpine ice have been identified, police in Switzerland announced on Tuesday.<SENT>The remains of Masa..."
1106243,2,5,3,0,0,"MADRID -- Spain's members of parliament have started voting on Socialist leader Pedro Sanchez' bid to form a new government, two months after the country held inconclusive elections.<SENT>The voti..."
15655,0,1,2,3,0,LOS ANGELES (AP) -- Jennifer Aniston and Justin Theroux are separating.<SENT>The couple says in a statement released through her publicist Thursday that the decision to split ''was mutual and lovi...
8346,0,6,4,2,0,"A gunman opened fire inside a gym at an upscale shopping mall outside Miami on Saturday afternoon, wounding two people before he was killed, Miami-Dade County police told local media.<SENT>The sho..."
9751,7,5,6,0,0,"CHAMPAIGN, Ill.<SENT>(AP) -- Relatives of a missing Chinese woman studying at the University of Illinois say they won't leave the country without her as the reward in the case rose to $50,000. Fam..."
...,...,...,...,...,...,...
1119352,0,0,2,2,0,"A Briton who was missing in Brussels since the bomb attacks in the Belgian capital has been confirmed as dead by the Foreign Office.<SENT>David Dixon, 53, from Nottingham, lived in Brussels with h..."
1011512,0,1,1,0,0,The Football Association has apologised for its own-goal in omitting one of England's most prolific scorers from a roll of honour at Wembley Stadium.<SENT>Bolton Wanderers legend Nat Lofthouse was...
694870,1,1,4,4,1,"The discovery of two bodies at a Nottinghamshire house is being treated as a double murder by police.<SENT>The property in Blenheim Close, Forest Town, was searched on 10 October after a tip-off.<..."


In [109]:
ref_sent_labels_df

,version,sent_idx,sentence,entry_id,deleted_label,add_above_label,add_below_label,edited_label,unchanged_label,refactored_label
0,2,0,"VIENNA (Reuters) - Austrian authorities are investigating reports of a man appearing in public in Adolf Hitler's birthplace as the Nazi dictator's double, including the distinctive mustache, hairc...",2841,0,0,1,0,1,0
1,2,1,"''I have often seen this gentlemen in Braunau and wonder if this means something,'' the Oberoesterreichische Nachrichten paper cited a local resident as saying on his Facebook page alongside a pic...",2841,0,1,0,1,0,0
2,2,2,"Hitler was born in Braunau am Inn, then part of Austria-Hungary, in 1889.",2841,0,0,0,0,1,0
3,2,3,Prosecutors confirmed the report.,2841,0,0,0,0,1,0
4,2,4,"The man, estimated to be 25 to 30 years old, was last seen in a local bookstore browsing through magazines about World War Two, adding he had identified himself in a local bar as ''Harald Hitler.",2841,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4,1,4,"Recent figures suggested almost 80,000 Scots were still without a NHS dentist.",301499,0,0,0,0,1,0
5,1,5,"NHS Grampian has more than 31,000 on its waiting list.",301499,0,0,0,1,0,0
6,1,6,Mr Newton said: ''I am sure there are a lot of people who want care but realise there is not any to have. ''They have heard there are so many on the waiting lists and they have thought they won't ...,301499,0,0,0,0,1,0
7,1,7,'' He said that as soon as more dentists started to reduce waiting lists more people would come forward to be seen.,301499,0,0,0,0,1,0


# Get Models to Run

In [5]:
import pickle
import pandas as pd 
from tensorboard_reducer import load_tb_events, reduce_events, write_csv, write_tb_events
import yaml
import glob

def get_events(version_num, path_to_ver):
    if isinstance(version_num, (str, int)):
        version_num = [version_num]    
    events_files = []
    for v in version_num:
        events_file = path_to_ver + '/version_%s' % v
        events_files.append(events_file)
    events = load_tb_events(events_files, handle_dup_steps='keep-last')
    return events

In [6]:
doc_hparams = {}
path_to_versions = '../modeling/tensorboard/old_architecture_docs/default'

version_num_to_notes = {}
for hparams_file in glob.glob(path_to_versions + '/*/hparams.yaml'):
    version_num = hparams_file.split('/')[-2]
    with open(hparams_file, "r") as stream:
        hparams = yaml.safe_load(stream)
        version_num_to_notes[version_num] = hparams.get('notes')
        doc_hparams[version_num] = hparams
        
version_map_s = pd.Series(version_num_to_notes)

pd.options.display.max_colwidth=200

(version_map_s
 .reset_index()
 .assign(index=lambda df: df['index'].str.split('_').str.get(1).astype(int)).sort_values('index')
 .dropna()
 .set_index('index')[0]
)

index
0                                              Roberta, Deleted binary, attention, more unfrozen
1                                              Roberta, Doc Add binary, attention, more unfrozen
2                                                           Roberta, Doc Edits binary, attention
3                                                        Roberta, Doc Refactor binary, attention
4                                              Roberta, Doc Add binary, attention, more unfrozen
5                                  Roberta, Deleted binary, attention, more unfrozen, no context
6                                               Roberta, Doc Edits binary, attention, no context
7                                  Roberta, Doc Add binary, attention, more unfrozen, no context
8                                            Roberta, Doc Refactor binary, attention, no context
9                                              Roberta, Doc Edits binary, CLS, no context, large
10                      

In [14]:
pd.set_option('display.max_columns', None)
hparam_df = pd.DataFrame(doc_hparams).T
hparam_df.iloc[0]#.apply(lambda x: set(x), axis=1)

accumulate_grad_batches      100
adam_beta1                   0.9
adam_beta2                 0.999
adam_epsilon                 0.0
batch_size                     1
                           ...  
warmup_steps                 0.0
weight_decay                   0
do_multitask                 NaN
do_version                   NaN
loss_weighting               NaN
Name: version_8, Length: 80, dtype: object

In [24]:
hparam_df.iloc[0].apply(type).value_counts()

<class 'int'>         25
<class 'bool'>        22
<class 'str'>         14
<class 'float'>       12
<class 'NoneType'>     5
<class 'list'>         2
Name: version_8, dtype: int64

In [26]:
hparam_df.iloc[0].loc[lambda s: s.apply(lambda x: isinstance(x, list))]

freeze_encoder_layers                                               []
label_to_idx_mapper      [refactored: (-1, 0], refactored: (-100, -1]]
Name: version_8, dtype: object

In [34]:
cols = list(filter(lambda x: x not in ['freeze_encoder_layers', 'label_to_idx_mapper'],  list(hparam_df.columns)))

In [54]:
hparam_variants = {}
for c in hparam_df.columns:
    if c not in ['label_to_idx_mapper', 'freeze_encoder_layers', 'num_output_tags', 'loss_weighting']:
        hparam_variants[c] = set(hparam_df[c].tolist())

In [58]:
import json

In [60]:
with open('/Users/alex/.cache/torch/transformers/named-models/roberta-base/config.json') as f:
    config = json.load(f)

In [57]:
ls /Users/alex/.cache/torch/transformers/named-models/roberta-base/

config.json              pytorch_model.bin        tokenizer_config.json
merges.txt               special_tokens_map.json  vocab.json


In [55]:
hparam_variants

{'accumulate_grad_batches': {10, 20, 100},
 'adam_beta1': {0.9},
 'adam_beta2': {0.999},
 'adam_epsilon': {1e-08},
 'batch_size': {1},
 'bidirectional': {False},
 'concat_headline': {False},
 'context_layer': {'lstm'},
 'contextual_layer_type': {'lstm'},
 'dataset_size': {27000, 135000},
 'discriminator_path': {None},
 'do_doc_pred': {True},
 'do_eval': {True},
 'do_train': {True},
 'doc_embed_arithmetic': {False},
 'dropout': {0.1},
 'embedding_dim': {768},
 'embedding_model_type': {'roberta'},
 'env': {'local'},
 'eval_data_file_s3': {None},
 'experiment': {'lstm_sequential'},
 'finetuned_lm_file': {None},
 'freeze_embedding_layer': {False},
 'freeze_pooling_layer': {False},
 'freeze_transformer': {False, True},
 'gradient_accumulation': {10, 20, 100},
 'hidden_dim': {512},
 'learning_rate': {1e-06},
 'local': {False},
 'log_all_metrics': {True},
 'log_interval': {10},
 'lstm_bidirectional': {False},
 'lstm_num_hidden_layers': {0, 2},
 'main_data_file': {'/job/.local/lib/python3.7/si

### Refactored Models

In [103]:
t1 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactored-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['freeze_transformer'] == True]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t1)

t2= (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactored-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t2)


t3 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactored-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == True]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t3)

t4 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactored-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
#  .loc[lambda df: df['do_version'] == False]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t4)

version_25    Roberta, Doc Refactor binary, CLS, LSTM no context, large, grad accum 10
Name: notes, dtype: object
version_38    Roberta, Doc Refactor binary, CLS no context, 1-6 frozen, large, grad accum 10
Name: notes, dtype: object
version_51    Roberta, Doc Refactor binary, CLS, 1-6 unfrozen, large, +Version
Name: notes, dtype: object
version_35    Roberta, Doc Refactor binary, CLS, 1-6 frozen, large, grad accum 10
Name: notes, dtype: object


In [104]:
t1 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('add-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['freeze_transformer'] == True]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t1)

t2= (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('add-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t2)


t3 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('add-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == True]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t3)

t4 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('add-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
#  .loc[lambda df: df['do_version'] == False]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t4)

version_24    Roberta, Doc Add binary, CLS, more unfrozen, LSTM, large, grad accum 10
Name: notes, dtype: object
version_29    Roberta, Doc Add binary, CLS, 1-6 frozen, no context, large, grad accum 10
Name: notes, dtype: object
version_50    Roberta, Doc Add binary, CLS, 1-6 unfrozen, grad accum 10, +Version
Name: notes, dtype: object
version_33    Roberta, Doc Add binary, CLS, 1-6 frozen, LSTM, large, grad accum 10
Name: notes, dtype: object


In [ ]:
trial-Roberta, Doc Add binary, CLS, more unfrozen, LSTM, large, grad accum 10, +Version__epoch=09-f1_macro=0.36.ckpt

trial-Roberta, Doc Add binary, CLS, more unfrozen, no context, large, grad accum 10__epoch=09-f1_macro=0.36.ckpt

trial-Roberta, Doc Add binary, CLS, more unfrozen, no context, large__epoch=09-f1_macro=0.30.ckpt

trial-Roberta, Doc Add binary, attention, more unfrozen, no context__epoch=09-f1_macro=0.32.ckpt

trial-Roberta, Doc Add binary, attention, more unfrozen__epoch=08-f1_macro=0.22.ckpt

In [107]:
t1 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('deleted-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['freeze_transformer'] == True]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t1)

t2= (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('deleted-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t2)


t3 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('deleted-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == True]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t3)

t4 = (hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('deleted-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
#  .loc[lambda df: df['do_version'] == False]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t4)

version_19    Roberta, Deleted binary, CLS, more unfrozen, LSTM, large, grad accum 10
Name: notes, dtype: object
version_28    Roberta, Deleted binary, CLS, 1-6 frozen, no context, large, grad accum 10
Name: notes, dtype: object
version_49    Roberta, Deleted binary, CLS, no context, 1-6 unfrozen, grad accum 10, +Version
Name: notes, dtype: object
version_32    Roberta, Deleted binary, CLS, 1-6 frozen, LSTM, large, grad accum 10
Name: notes, dtype: object


In [110]:
doc_hparams = {}
path_to_versions = '../modeling/tensorboard/old_architecture/default'

version_num_to_notes = {}
for hparams_file in glob.glob(path_to_versions + '/*/hparams.yaml'):
    version_num = hparams_file.split('/')[-2]
    with open(hparams_file, "r") as stream:
        hparams = yaml.safe_load(stream)
        version_num_to_notes[version_num] = hparams.get('notes')
        doc_hparams[version_num] = hparams
        
version_map_s = pd.Series(version_num_to_notes)

pd.options.display.max_colwidth=200

(version_map_s
 .reset_index()
 .assign(index=lambda df: df['index'].str.split('_').str.get(1).astype(int)).sort_values('index')
 .dropna()
 .set_index('index')[0]
)

index
0                         Roberta, Sentence Operations binary, high level labels
1                         Roberta, Sentence Operations binary, high level labels
2                                Roberta, Sentence Add binary, high level labels
3                         Roberta, Sentence Operations binary, high level labels
4                                Roberta, Sentence Add binary, high level labels
5                         Roberta, Sentence Operations binary, high level labels
6                           Roberta, Sentence Refactor binary, high level labels
7                                Roberta, Sentence Add binary, high level labels
8                                Roberta, Sentence Add binary, high level labels
9                         Roberta, Sentence Operations binary, high level labels
10                          Roberta, Sentence Refactor binary, high level labels
11                                Roberta, Sentence Operations binary, attention
12                 Rob

In [122]:
t1 = (sent_hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactor-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['freeze_transformer'] == True]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t1)

t2= (sent_hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactor-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 0]
 .loc[lambda df: df['do_version'] == False]
 ['notes']
)
print(t2)


t3 = (sent_hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactor-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['do_version'] == True]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t3)

t4 = (sent_hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactor-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['do_version'] == False]
 .loc[lambda df: df['freeze_transformer'] == False]
 ['notes']
)
print(t4)

Series([], Name: notes, dtype: object)
version_27    Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, -LSTM
Name: notes, dtype: object
version_22    Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, +Version
version_21    Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, +Version
Name: notes, dtype: object
version_38    Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, +LSTM
Name: notes, dtype: object


In [131]:
pd.options.display.max_rows = 200

In [132]:
(sent_hparam_df
 .loc[lambda df: df['train_data_file_s3'].str.contains('refactor-labels')]
 .loc[lambda df: df['num_contextual_layers'] == 2]
 .loc[lambda df: df['do_version'] == True]
 .loc[lambda df: df['freeze_transformer'] == False].T
#  ['notes']
)

,version_22,version_21
accumulate_grad_batches,100,100
adam_beta1,0.9,0.9
adam_beta2,0.999,0.999
adam_epsilon,0.0,0.0
batch_size,1,1
bidirectional,False,False
concat_headline,False,False
context_layer,lstm,lstm
contextual_layer_type,lstm,lstm
dataset_size,45000,45000


In [125]:
#  trial-Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, +LSTM__epoch=09-f1_macro=0.50.ckpt
#  trial-Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, +LSTM__epoch=09-f1_macro=0.54.ckpt
#  trial-Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, +Version__epoch=04-f1_macro=0.51.ckpt
#  trial-Roberta, Sentence Refactor binary, attention, 1-6 Unfrozen, -LSTM__epoch=00-f1_macro=0.50.ckpt
#  trial-Roberta, Sentence Refactor binary, attention__epoch=00-f1_macro=0.50.ckpt
#  trial-Roberta, Sentence Refactor binary, attention__epoch=02-f1_macro=0.50.ckpt
#  trial-Roberta, Sentence Refactor binary, high level labels__epoch=00-f1_macro=0.50.ckpt
#  trial-Roberta, Sentence Refactor binary, high level labels__epoch=08-f1_macro=0.59.ckpt

In [3]:
sent_hparam_df = pd.DataFrame(doc_hparams).T

NameError: name 'doc_hparams' is not defined

In [115]:
sent_hparam_df['train_data_file_s3']

version_8           data/sentence-edits__add-labels.tsv
version_36    data/sentence-edits__operation-labels.tsv
version_31          data/sentence-edits__add-labels.tsv
version_38     data/sentence-edits__refactor-labels.tsv
version_6      data/sentence-edits__refactor-labels.tsv
version_1            data/sentence-operation-labels.tsv
version_0            data/sentence-operation-labels.tsv
version_39          data/sentence-edits__add-labels.tsv
version_7           data/sentence-edits__add-labels.tsv
version_30          data/sentence-edits__add-labels.tsv
version_9            data/sentence-operation-labels.tsv
version_37          data/sentence-edits__add-labels.tsv
version_15          data/sentence-edits__add-labels.tsv
version_12           data/sentence-operation-labels.tsv
version_24    data/sentence-edits__operation-labels.tsv
version_23          data/sentence-edits__add-labels.tsv
version_22     data/sentence-edits__refactor-labels.tsv
version_25          data/sentence-edits__add-lab

In [134]:
ls ../modeling/data/edits__docs-ref-*

../modeling/data/edits__docs-ref-baseline.txt
../modeling/data/edits__docs-ref-partially-unfrozen-no-context.txt


In [360]:
ref_baseline = open('../modeling/data/edits__docs-ref-baseline.txt').read().split('\n')
ref_unfrozen = open('../modeling/data/edits__docs-ref-partially-unfrozen-no-context.txt').read().split('\n')

In [171]:
# f.write(str(pred_tags))
# f.write(label_col.to_json()

In [362]:
t= list(map(lambda x: x.replace('[', '').split(']'), ref_baseline[1::2]))
t_df = pd.DataFrame(t, columns=['pred', 'true'])
t_df['true'] = t_df['true'].astype(int).pipe(lambda s: s > 0).astype(int)

In [363]:
from sklearn.metrics import f1_score
f1_score(t_df['true'].astype(str), t_df['pred'].astype(str), average='micro')

0.6455

In [364]:
f1_score(t_df['true'].astype(str), t_df['pred'].astype(str), average='macro')

0.3922819811607414

In [365]:
partially_unfrozen = pd.DataFrame(
    list(map(lambda x: x.replace('[', '').split(']'), ref_unfrozen[1::2])), 
    columns=['pred', 'true']
)

In [366]:
partially_unfrozen['true'] = partially_unfrozen['true'].astype(int).pipe(lambda s: s > 0).astype(int).astype(str)

In [367]:
f1_score(partially_unfrozen['true'], partially_unfrozen['pred'], average='micro')

0.6455

In [368]:
f1_score(partially_unfrozen['true'], partially_unfrozen['pred'], average='macro')

0.3922819811607414

In [379]:
f1_score(partially_unfrozen['true'], ['0'] * 2000, average='micro')

0.6455

In [380]:
f1_score(partially_unfrozen['true'], ['0'] * 2000, average='macro')

0.3922819811607414

In [381]:
f1_score(partially_unfrozen['true'], np.random.choice(['0', '1', '-1'], 2000), average='macro')

0.2579447323290069

In [382]:
f1_score(partially_unfrozen['true'], np.random.choice(['0', '1', '-1'], 2000), average='micro')

0.3415

# Evaluate sentence ref data

In [221]:
import ast

In [194]:
ls ../modeling/data/edits__sents-ref-*

../modeling/data/edits__sents-ref-context.txt
../modeling/data/edits__sents-ref-partially-unfrozen.txt
../modeling/data/edits__sents-ref-version-with-context.txt


In [331]:
b = open('../modeling/data/edits__sents-ref-version-with-context.txt').read().split('\n')

In [330]:
b = list(filter(lambda x: 'refactored_label' in x, b))

In [ ]:
df = pd.DataFrame(list(map(lambda x: x.split(']{'), b)))
df[1] = df[1].str.split(':').str.get(1).str.replace('}', '', regex=False).apply(ast.literal_eval)
df[0] = df[0].pipe(lambda s: s + ']').apply(ast.literal_eval)
df[1] = df[1].apply(lambda x: list(map(lambda y: int(y == 0), x)))

In [ ]:
y_pred = [j2 for i2 in df[0].tolist() for j2 in i2]
y_true = [j2 for i1 in df[1].tolist() for j2 in i1]

In [320]:
t = pd.DataFrame([y_pred, y_true]).T

In [321]:
w = t[1].value_counts().pipe(lambda s: 1/s)

In [339]:
b_t = (t
 .assign(w=lambda df: df.merge(w.to_frame('w'), right_index=True, left_on=1)['w'])
 .pipe(lambda df: df.sample(300, weights=df['w']))
)

In [323]:
f1_score(b_t[1].astype(str), b_t[0].astype(str), average='micro')

0.572

In [354]:
import numpy as np 

In [358]:
f1_score(b_t[1].astype(str), np.random.choice(['0', '1', '-1'], size=300), average='macro')

0.2470896529818153

In [359]:
f1_score(b_t[1].astype(str).tolist(), np.random.choice(['0', '1', '-1'], size=300), average='micro')

0.34

In [348]:
f1_score(b_t[1].astype(str).tolist(), ['1'] * 300, average='macro')

0.3478260869565218

In [210]:
training_df = pd.read_csv('../../controlled-sequence-gen/data/sentence-edits__refactor-labels.tsv', sep='\t', header=None)

In [213]:
training_df[0].sort_values().unique()

array(['refactor_down', 'refactor_unchanged'], dtype=object)